In [1]:
from __future__ import print_function

import os
import re
import cPickle, gzip
import progressbar

from rdkit import Chem
from rdkit.Chem import AllChem
import parser.Smipar as Smipar

In [2]:
with gzip.open('gen_rxn/data/all_rxns.pkl.gz', 'rb') as f:
    rxn_all = cPickle.load(f)

data_length = len(rxn_all)
print(data_length)

865118


In [3]:
# data reloader

with gzip.open('data/vocab/vocab_dict.pkl.gz', 'rb') as dict_file:
    vocab_reactants, vocab_products = cPickle.load(dict_file)

with gzip.open('data/vocab/vocab_list.pkl.gz', 'rb') as list_file:
    reactants_token_list, products_token_list = cPickle.load(list_file)

In [4]:
def cano(smiles): # canonicalize smiles by MolToSmiles function
    return Chem.MolToSmiles(Chem.MolFromSmiles(smiles)) if (smiles != '') else ''

In [5]:
bar = progressbar.ProgressBar(max_value=data_length)

token_len = []

with gzip.open('data/training_data(gen_rxn).pkl.gz', 'wb') as training_file:
    
    for i, rsmi in enumerate(rxn_all):

        reactant_list = []
        agent_list = []
        product_list = []

        try:
            split_rsmi = rsmi.split('>')
            reactants = cano(split_rsmi[0]).split('.')
            agents = cano(split_rsmi[1]).split('.')
            products = cano(split_rsmi[2]).split('.')

            for reactant in reactants:
                reactant_list += Smipar.parser_list(reactant)
                reactant_list += '.'
            for agent in agents:
                agent_list += Smipar.parser_list(agent)
                agent_list += '.'
            for product in products:
                product_list += Smipar.parser_list(product)
                product_list += '.'

            reactant_list.pop() # to pop last '.'
            agent_list.pop()
            product_list.pop()

            reactant_list += '>'
            reactant_list += agent_list

            token_len.append((len(reactant_list), len(product_list)))

            cPickle.dump(([reactants_token_list.index(r) for r in reactant_list],
                          [products_token_list.index(p) for p in product_list],
                          'gen'), training_file, 2)
        
        except:
            pass
                     
        bar.update(i)
        
bar.finish()

100% (865118 of 865118) |##################| Elapsed Time: 1:43:46 Time: 1:43:46


In [6]:
trans = zip(*token_len)
print(max(trans[0]), max(trans[1]))

75 47


In [7]:
count=0
bar = progressbar.ProgressBar(max_value=len(token_len))
with gzip.open('data/training_data(gen_rxn).pkl.gz', 'rb') as data_file:
    with gzip.open('data/train(gen_rxn).pkl.gz', 'wb') as train_file:
        with gzip.open('data/dev(gen_rxn).pkl.gz', 'wb') as dev_file:
            i = 1
            while 1:
                try:
                    reactants, products, _ = cPickle.load(data_file)
                    if i%100:
                        cPickle.dump((reactants, products), train_file, 2)
                    else:
                        cPickle.dump((reactants, products), dev_file, 2)
                        count += 1
                except EOFError:
                    break
                bar.update(i)
                i += 1

bar.finish()
print(count, len(token_len)-count)

 99% (863977 of 864378) |################## | Elapsed Time: 0:02:04 ETA: 0:00:00

8643 855735


100% (864378 of 864378) |##################| Elapsed Time: 0:02:04 Time: 0:02:04
